# FINAL PROJECT

**Names:** -- Diana Zepeda Tatengo and Victor Hugo Gomez Soto --

**e-mails:** -- diana.zepeda6085@alumnos.udg.mx and victor.gomez2701@alumnos.udg.mx --

# MODULES

In [19]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# WIDGETS

In [ ]:
# Carga de archivo
upload = widgets.FileUpload(accept='.csv,.xlsx', multiple=False)

# Selectors
column_selector = widgets.SelectMultiple(options=[], description="Columns:", layout=widgets.Layout(width='100%'))
compare_with_selector = widgets.Dropdown(options=[], description="Compare with:", layout=widgets.Layout(width='100%'))
date_range = widgets.SelectionRangeSlider(
    description='Range:',
    options=[datetime(2000, 1, 1), datetime(2000, 12, 31)],
    value=(datetime(2000, 1, 1), datetime(2000, 12, 31)),
    layout=widgets.Layout(width='100%'),
    continuous_update=False
)
# Outputs
output_info = widgets.Output()
output_table = widgets.Output()
output_plot = widgets.Output()

# Global
df_global = None

# Función de carga
def on_file_upload(change):
    global df_global
    output_info.clear_output()
    output_table.clear_output()
    output_plot.clear_output()

    if upload.value:
        uploaded_file = upload.value[0]
        content = uploaded_file['content']
        filename = uploaded_file['name']


        try:
            if filename.endswith('.csv'):
                df = pd.read_csv(pd.io.common.BytesIO(content))
            else:
                df = pd.read_excel(pd.io.common.BytesIO(content))

            df.columns = df.columns.str.strip()

            if all(col in df.columns for col in ["Year", "Month", "Day"]):
                df["Fecha"] = pd.to_datetime(df[["Year", "Month", "Day"]])
                df = df.sort_values("Fecha")
            else:
                raise ValueError("The columns 'Year', 'Month' y 'Day' are required.")

            df_global = df

            columnas_numericas = df.select_dtypes(include='number').columns.tolist()
            columnas_numericas = [col for col in columnas_numericas if col not in ["Year", "Month", "Day"]]
            column_selector.options = columnas_numericas
            compare_with_selector.options = columnas_numericas
            if columnas_numericas:
                column_selector.value = tuple(columnas_numericas[:1])
                compare_with_selector.value = columnas_numericas[1] if len(columnas_numericas) > 1 else None

            fechas_unicas = df["Fecha"].sort_values().unique()
            date_range.options = list(fechas_unicas)
            date_range.value = (fechas_unicas[0], fechas_unicas[-1])

            with output_info:
                print(f"✅ File Uploaded: {filename}")
        except Exception as e:
            with output_info:
                print(f"❌ Error: {e}")

def visualizar_datos(*args):
    output_table.clear_output()
    output_plot.clear_output()

    if df_global is None or not column_selector.value:
        return

    columnas = list(column_selector.value)
    comparar_con = compare_with_selector.value

    try:
        df = df_global[["Fecha"] + columnas].copy().dropna()
        if comparar_con:
            df[comparar_con] = df_global[comparar_con]

        fecha_inicio, fecha_fin = date_range.value
        df = df[(df["Fecha"] >= fecha_inicio) & (df["Fecha"] <= fecha_fin)]

        with output_table:
            display(df.head(10))

        with output_plot:
             # Temporal Series
            plt.figure(figsize=(12, 4))
            for col in columnas:
                plt.plot(df["Fecha"], df[col], label=col)
            if comparar_con:
                plt.plot(df["Fecha"], df[comparar_con], label=comparar_con, linestyle="--")
            plt.title("Temporal Series")
            plt.xlabel("Date")
            plt.ylabel("Value")
            plt.legend()
            plt.grid(True)
            plt.tight_layout()
            plt.show()

            # Create a grid of 2 rows and 2 columns for the subplots.
            fig, axs = plt.subplots(2, 2, figsize=(12, 10))

            # Adjust the spacing between the subplots.
            plt.subplots_adjust(hspace=0.3, wspace=0.3)                              

            # Correlation Heatmap
            sns.heatmap(df[columnas + ([comparar_con] if comparar_con else [])].corr(), annot=True, cmap="coolwarm", ax=axs[0, 0])
            axs[0, 0].set_title("Correlation Heatmap")

            # Scatter plot if there is a comparison    
            if comparar_con and len(columnas) == 1:
                axs[0, 1].scatter(df[columnas[0]], df[comparar_con], alpha=0.5)
                axs[0, 1].set_title(f"Scatter: {columnas[0]} vs {comparar_con}")
                axs[0, 1].set_xlabel(columnas[0])
                axs[0, 1].set_ylabel(comparar_con)
                axs[0, 1].grid(True)        

            # Boxplot
            sns.boxplot(y=df[columnas[0]], color="skyblue", ax=axs[1, 0])
            axs[1, 0].set_title(f"Box Plot of {columnas[0]}")

            # Bars by month (if possible)
            df["Mes"] = df["Fecha"].dt.month
            df.groupby("Mes")[columnas[0]].mean().plot(kind="bar", ax=axs[1, 1])
            axs[1, 1].set_title(f"{columnas[0]} Monthly Average")
            axs[1, 1].set_xlabel("Month")
            axs[1, 1].set_ylabel("Average Value")
            axs[1, 1].grid(True)

            # Adjust and display
            plt.tight_layout()
            plt.show()
          
    except Exception as e:
        with output_plot:
            print(f"❌ Plotting Error: {e}")


# Eventos
upload.observe(on_file_upload, names='value')
column_selector.observe(visualizar_datos, names='value') 
compare_with_selector.observe(visualizar_datos, names='value') 
date_range.observe(visualizar_datos, names='value')

# Panel-style Layout: Columns
sidebar = widgets.VBox([
    widgets.Label("📁 Upload File"),  
    upload,
    output_info,  
    widgets.Label("📌 Columns to display"),
    column_selector,
    widgets.Label("📈 Compare With"),
    compare_with_selector,
    widgets.Label("📅 Date Range"),
    date_range
], layout=widgets.Layout(width='30%', padding='10px'))

main_display = widgets.VBox([
    output_plot,
    widgets.Label("📋 Data Table"),
    output_table
], layout=widgets.Layout(width='70%', padding='10px'))

# DASHBOARD

In [21]:
# Header
display(HTML("""
<div style="background-color:#1976D2; padding: 10px 20px; color: white; font-size: 24px; font-family: Arial; text-align: center;">
    🌍📊🌦️<b>Climate Data Dashboard</b>🌡️🌞🌧️
</div>
"""))

# Display widgets
display(widgets.HBox([sidebar, main_display]))